In [ ]:
import pandas as pd

file_name = 'results_gmean_all.xlsx'
# 读取Excel文件
file_path = 'C:\\Users\\zsc\\Desktop\\finall_result\\' + file_name  # 请替换为你的Excel文件路径
xls = pd.ExcelFile(file_path)

# 为每个工作表创建DataFrame
dataframes = {}
for sheet_name in xls.sheet_names:
    # 读取工作表数据，跳过第一行（标题行）
    df = pd.read_excel(xls, sheet_name=sheet_name, header=0)

    # 将DataFrame存储在字典中，键为工作表名称
    dataframes[sheet_name] = df

    # 也可以直接创建同名变量（不推荐，但如果你确实需要）
    # globals()[sheet_name] = df

# 现在你可以通过以下方式访问DataFrame：
# 1. 通过字典访问（推荐）
# print(dataframes['数据集1名称'].head())

# 2. 或直接通过变量名访问（如果上面使用了globals()）
# print(数据集1名称.head())

In [ ]:
import pandas as pd
from scipy.stats import wilcoxon

file_name = 'results_gmean_all.xlsx'
# 读取Excel文件
file_path = 'C:\\Users\\zsc\\Desktop\\finall_result\\' + file_name  # 请替换为你的Excel文件路径
# 读取 Excel 文件
xls = pd.ExcelFile(file_path)

# 存储原始数据的 DataFrame
dataframes = {}
# 存储 Wilcoxon 检验结果的 DataFrame
dataframes_Wilcoxon = {}

for sheet_name in xls.sheet_names:
    # 读取数据
    df = pd.read_excel(xls, sheet_name=sheet_name)
    dataframes[sheet_name] = df

    # 准备存储检验结果
    results = {
        'Algorithm': [],
        'Statistic': [],
        'p-value': [],
        'Significant (p < 0.05)': []
    }

    # 第 10 种算法（MILE）作为基准
    baseline_algo = df.columns[-1]  # 最后一列（MILE）
    baseline_data = df[baseline_algo]

    # 对前 9 种算法分别做检验
    for algo in df.columns[:-1]:  # 遍历除最后一列外的所有算法
        algo_data = df[algo]

        # 执行 Wilcoxon 检验（假设数据是配对的）
        stat, p_value = wilcoxon(algo_data, baseline_data)

        # 记录结果
        results['Algorithm'].append(algo)
        results['Statistic'].append(stat)
        results['p-value'].append(p_value)
        results['Significant (p < 0.05)'].append(p_value < 0.05)

    # 将结果转为 DataFrame 并存储
    results_df = pd.DataFrame(results)
    dataframes_Wilcoxon[sheet_name] = results_df

# 示例：查看第一个数据集的检验结果
first_dataset = list(dataframes_Wilcoxon.keys())[0]
print(f"Wilcoxon 检验结果 - {first_dataset}:")
print(dataframes_Wilcoxon[first_dataset])

In [ ]:
import pandas as pd
from scipy.stats import wilcoxon

# file_name = 'results_gmean_all.xlsx'
file_name = 'results_mauc_all.xlsx'
# 读取Excel文件
file_path = 'C:\\Users\\zsc\\Desktop\\finall_result\\' + file_name  # 请替换为你的Excel文件路径
xls = pd.ExcelFile(file_path)

# 存储原始数据的 DataFrame
dataframes = {}
# 存储 Wilcoxon 检验结果的 DataFrame
dataframes_Wilcoxon = {}

for sheet_name in xls.sheet_names:
    # 读取数据
    df = pd.read_excel(xls, sheet_name=sheet_name)
    dataframes[sheet_name] = df

    # 准备存储检验结果
    results = {
        'Algorithm': [],
        'Statistic': [],
        'p-value': [],
        'Significant (p < 0.05)': []
    }

    # 第 10 种算法（MILE）作为基准
    baseline_algo = df.columns[-1]  # 最后一列（MILE）
    baseline_data = df[baseline_algo]

    # 对前 9 种算法分别做检验
    for algo in df.columns[:-1]:  # 遍历除最后一列外的所有算法
        algo_data = df[algo]

        try:
            # 执行 Wilcoxon 检验（假设数据是配对的）
            stat, p_value = wilcoxon(algo_data, baseline_data)
            significant = p_value < 0.05
        except ValueError as e:
            # 如果所有差值均为 0，则返回默认值
            stat, p_value = 0, 1.0
            significant = False

        # 记录结果
        results['Algorithm'].append(algo)
        results['Statistic'].append(stat)
        results['p-value'].append(p_value)
        results['Significant (p < 0.05)'].append(significant)

    # 将结果转为 DataFrame 并存储
    results_df = pd.DataFrame(results)
    dataframes_Wilcoxon[sheet_name] = results_df

# 遍历所有数据集，输出 Wilcoxon 检验结果
for dataset_name, results_df in dataframes_Wilcoxon.items():
    print(f"\n=== Wilcoxon 检验结果 - {dataset_name} ===")
    print(results_df)
    print("=" * 50)  # 分隔线

## Wilcoxon 秩和检验

In [1]:
import pandas as pd
from scipy import stats
from scipy.stats import wilcoxon

# file_name = 'results_gmean_all.xlsx'
file_name = 'results_mauc_all.xlsx'
# 读取Excel文件
file_path = 'C:\\Users\\zsc\\Desktop\\finall_result\\' + file_name  # 请替换为你的Excel文件路径
xls = pd.ExcelFile(file_path)

# 存储原始数据的 DataFrame
dataframes = {}
# 存储 Wilcoxon 检验结果的 DataFrame
dataframes_Wilcoxon = {}

for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name) * 100
    baseline_algo = df.columns[-1]  # MILE
    baseline_data = df[baseline_algo]

    results = {
        'Algorithm': [],
        'Statistic': [],
        'p-value': [],
        'Comparison with MILE': []  # 新增：标注优劣关系
    }

    for algo in df.columns[:-1]:
        algo_data = df[algo]

        try:
            stat, p_value = stats.mannwhitneyu(algo_data, baseline_data,alternative='less')
        except ValueError:
            stat, p_value = 0, 1.0

        # 判断优劣关系
        if p_value >= 0.05:
            comparison = "无足够证据表明小于MILE"
        else:
            # if stat < (len(algo_data) * (len(algo_data) + 1) / 4):  # 理论中值
            #     comparison = "显著小于MILE"
            # else:
            #     comparison = "显著优于MILE"
            comparison = "显著小于MILE"

        results['Algorithm'].append(algo)
        results['Statistic'].append(stat)
        results['p-value'].append(p_value)
        results['Comparison with MILE'].append(comparison)

    dataframes_Wilcoxon[sheet_name] = pd.DataFrame(results)

# 输出所有数据集的比较结果
for dataset_name, results_df in dataframes_Wilcoxon.items():
    print(f"\n=== 算法对比结果 - {dataset_name} ===")
    print(results_df)
    print("=" * 60)


=== 算法对比结果 - Balance_Scale ===
         Algorithm  Statistic       p-value Comparison with MILE
0              NOS      150.0  1.104056e-06             显著小于MILE
1              RUS        0.0  1.509930e-11             显著小于MILE
2              ROS      457.5  5.470916e-01        无足够证据表明小于MILE
3            SMOTE        0.0  1.509930e-11             显著小于MILE
4  BorderlineSMOTE        0.0  1.509930e-11             显著小于MILE
5     KMeansSMOTE         8.0  3.347759e-11             显著小于MILE
6         E-MOSAIC      416.0  3.102019e-01        无足够证据表明小于MILE
7              SPE       66.0  7.143798e-09             显著小于MILE
8              BRF        0.0  1.508983e-11             显著小于MILE

=== 算法对比结果 - Dermatology ===
         Algorithm  Statistic       p-value Comparison with MILE
0              NOS       30.0  1.679649e-11             显著小于MILE
1              RUS        0.0  1.508038e-11             显著小于MILE
2              ROS       70.0  1.006703e-08             显著小于MILE
3            SMOTE        8.